In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

### Make scene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.panda, None,
                INDY_IP)]
              , connection_list=[False])
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# # deprecated: s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"panda0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

connection command:
panda0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]


## Preapare background

In [3]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.3,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="panda0_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
grip0 = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

Please create a subscriber to the marker


## Drawer

In [4]:
from pkg.utils.code_scraps import add_drawer
drawer_bot, drawer, handle = add_drawer(pscene, center=(1,0,0.5))

### prepare planner

In [5]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


In [6]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
mplan.motion_filters = [GraspChecker(pscene), ReachChecker(pscene)]
mplan.incremental_constraint_motion = True

### Plan

In [7]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

   Use a production WSGI server instead.
 * Debug mode: off
['drawer', 'handle']
(0, 'drawer_bot')


In [8]:
goal_nodes = [(2,'drawer_bot')]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

Use 18/36 agents
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
try: 0 - (0, 'drawer_bot')->(0, 'grip0')
try transition motion
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
try: 0 - (0, 'drawer_bot')->(0, 'grip0')
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
try transition motion
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (0, 'drawer_bot')->(1, 'drawer_bot')
try: 0 - (0, 'drawer_bot')->(0, 'grip0')
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (0, 'drawer_bot')->(0, 'grip0')
try transition motion
Motion Filer Failure: GraspChecker
resu

collision
result: 3 - (0, 'grip0')->(1, 'grip0') = success
branching: 1->15 (0.58/30.0 s, steps/err: 1(21.6300487518 ms)/0)
result: 7 - (1, 'grip0')->(2, 'grip0') = fail
branching: 3->17 (0.59/30.0 s, steps/err: 1(11.9700431824 ms)/0)
result: 4 - (0, 'grip0')->(1, 'grip0') = success
try: 4 - (0, 'grip0')->(1, 'grip0')
try transition motion
branching: 0->16 (0.59/30.0 s, steps/err: 48(234.399080276 ms)/0.00195686355417)
result: 0 - (0, 'drawer_bot')->(0, 'grip0') = success
Motion Plan Failure
constrained motion tried: False
result: 11 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
branching: 0->19 (0.61/30.0 s, steps/err: 33(242.64383316 ms)/0.00146395946202)
result: 10 - (1, 'grip0')->(2, 'grip0') = fail
Motion Plan Failure
try: 11 - (0, 'drawer_bot')->(0, 'grip0')
try: 4 - (0, 'grip0')->(1, 'grip0')
try: 4 - (0, 'grip0')->(1, 'grip0')
try: 15 - (1, 'grip0')->(2, 'grip0')
result: 7 - (1, 'grip0')->(2, 'grip0') = fail
try: 5 - (0, 'grip0')->(0, 'drawer_bot')
branching: 4->18 (0.61/30.0 s

try: 30 - (1, 'grip0')->(2, 'grip0')
end
branching: 14->36 (0.83/30.0 s, steps/err: 21(185.667991638 ms)/0.000179045486483)
constrained motion tried: True
end
constrained motion tried: True
result: 13 - (1, 'grip0')->(2, 'grip0') = success
joint min
result: 15 - (1, 'grip0')->(2, 'grip0') = success
try: 36 - (2, 'grip0')->(2, 'drawer_bot')
branching: 13->37 (0.84/30.0 s, steps/err: 21(311.975955963 ms)/0.000179045486483)
result: 0 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
branching: 15->38 (0.84/30.0 s, steps/err: 21(202.063083649 ms)/0.000179045486483)
try: 38 - (2, 'grip0')->(2, 'drawer_bot')
constrained motion tried: False
try transition motion
joint min
try: 37 - (2, 'grip0')->(2, 'drawer_bot')
constrained motion tried: False
Motion Plan Failure
result: 32 - (1, 'grip0')->(2, 'grip0') = fail
try: 37 - (2, 'grip0')->(2, 'drawer_bot')
Motion Plan Failure
result: 24 - (1, 'grip0')->(2, 'grip0') = fail
Goal reached
try: 25 - (0, 'grip0')->(1, 'grip0')
transition motion tried: False

try: 14 - (1, 'grip0')->(2, 'grip0')
Goal reached
transition motion tried: False
Motion Plan Failure
result: 3 - (0, 'grip0')->(0, 'drawer_bot') = fail
try: 28 - (0, 'drawer_bot')->(1, 'drawer_bot')
result: 28 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 26 - (0, 'grip0')->(0, 'drawer_bot')
end
constrained motion tried: True
result: 30 - (1, 'grip0')->(2, 'grip0') = success
branching: 30->57 (1.29/30.0 s, steps/err: 21(160.187959671 ms)/0.000206446624328)
result: 26 - (0, 'grip0')->(0, 'drawer_bot') = success
Goal reached
branching: 26->58 (1.3/30.0 s, steps/err: 1(7.13014602661 ms)/0)
try: 57 - (2, 'grip0')->(2, 'drawer_bot')
try: 58 - (0, 'drawer_bot')->(1, 'drawer_bot')
try transition motion
transition motion tried: False
Motion Plan Failure
result: 34 - (2, 'grip0')->(2, 'drawer_bot') = fail
result: 58 - (0, 'drawer_bot')->(1, 'drawer_bot') = fail
Not available:drawer-0
np_exclude:['h1']
bd_exclude:drawer_handle
try: 43 - (1, 'grip0')->(2, 'grip0')
============== Non-availabl

Motion Plan Failure
result: 40 - (0, 'grip0')->(0, 'drawer_bot') = fail
======================= terminated 1: Stop called from other agent  (1.7/30.0) ===============================
transition motion tried: False
Motion Plan Failure
result: 57 - (2, 'grip0')->(2, 'drawer_bot') = fail
======================= terminated 17: Stop called from other agent  (1.8/30.0) ===============================
========================== FINISHED (1.9 / 30.0 s) ==============================]


In [9]:
snode_schedule = tplan.get_best_schedule()
ppline.play_schedule(snode_schedule)

(0, 'drawer_bot')->(0, 'grip0')
(0, 'grip0')->(1, 'grip0')
(1, 'grip0')->(2, 'grip0')
(2, 'grip0')->(2, 'drawer_bot')
(2, 'drawer_bot')->(2, 'drawer_bot')


### push back

In [10]:
gscene.show_pose(crob.home_pose)
term_state = snode_schedule[-1].state
print(pscene.subject_name_list)
print(term_state.node)

['drawer', 'handle']
(2, 'drawer_bot')


In [11]:
goal_nodes = [(1,'drawer_bot')]
ppline.search(term_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

Use 18/36 agents
try: 0 - (2, 'drawer_bot')->(2, 'drawer_bot')
try: 0 - (2, 'drawer_bot')->(2, 'drawer_bot')
try: 0 - (2, 'drawer_bot')->(2, 'grip0')
Motion Filer Failure: GraspChecker
try: 0 - (2, 'drawer_bot')->(2, 'drawer_bot')
result: 0 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
result: 0 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 0 - (2, 'drawer_bot')->(2, 'drawer_bot')
try: 0 - (2, 'drawer_bot')->(2, 'grip0')
result: 0 - (2, 'drawer_bot')->(2, 'grip0') = fail
try: 0 - (2, 'drawer_bot')->(2, 'grip0')
result: 0 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
Motion Filer Failure: GraspChecker
result: 0 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
result: 0 - (2, 'drawer_bot')->(2, 'grip0') = fail
try transition motion
try: 0 - (2, 'drawer_bot')->(2, 'drawer_bot')
try: 0 - (2, 'drawer_bot')->(2, 'drawer_bot')
try: 0 - (2, 'drawer_bot')->(2, 'drawer_bot')
result: 0 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
result: 0 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
result: 

branching: 8->9 (0.75/30.0 s, steps/err: 1(33.5729122162 ms)/0)
try: 3 - (2, 'grip0')->(2, 'drawer_bot')
np_exclude:['h1']
constrained motion tried: True
bd_exclude:drawer_handle
try transition motion
result: 1 - (2, 'grip0')->(1, 'grip0') = success
branching: 1->10 (0.8/30.0 s, steps/err: 21(277.776956558 ms)/0.000327244050482)
result: 9 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
collision
try: 10 - (1, 'grip0')->(1, 'drawer_bot')
constrained motion tried: False
============== Non-available transition: sample again =====================
try: 8 - (2, 'grip0')->(2, 'grip0')
try: 0 - (2, 'drawer_bot')->(2, 'grip0')
Motion Plan Failure
try transition motion
result: 2 - (2, 'grip0')->(1, 'grip0') = fail
try transition motion
end
constrained motion tried: True
collision
Not available:drawer-0
result: 1 - (2, 'grip0')->(1, 'grip0') = success
constrained motion tried: False
np_exclude:['h1']
Motion Plan Failure
branching: 1->11 (0.87/30.0 s, steps/err: 21(273.203134537 ms)/0.00032724405048

try: 11 - (1, 'grip0')->(2, 'grip0')
branching: 9->26 (1.25/30.0 s, steps/err: 6(69.0488815308 ms)/0.00138968799259)
result: 0 - (2, 'drawer_bot')->(2, 'grip0') = success
transition motion tried: True
try: 25 - (2, 'drawer_bot')->(2, 'drawer_bot')
branching: 0->27 (1.26/30.0 s, steps/err: 36(273.427009583 ms)/0.00180626458034)
result: 25 - (2, 'drawer_bot')->(1, 'drawer_bot') = fail
try: 9 - (2, 'drawer_bot')->(2, 'grip0')
result: 7 - (2, 'drawer_bot')->(2, 'grip0') = success
try: 18 - (2, 'grip0')->(2, 'grip0')
branching: 7->28 (1.28/30.0 s, steps/err: 126(316.411018372 ms)/0.0012723587059)
Motion Filer Failure: GraspChecker
collision
Goal reached
transition motion tried: False
try: 17 - (2, 'grip0')->(2, 'drawer_bot')
Motion Plan Failure
constrained motion tried: False
Motion Plan Failure
result: 2 - (2, 'grip0')->(1, 'grip0') = fail
result: 9 - (2, 'drawer_bot')->(2, 'grip0') = fail
======================= terminated 2: required answers acquired  (1.3/30.0) =========================

======================= terminated 5: Stop called from other agent  (1.8/30.0) ===============================
transition motion tried: False
Motion Plan Failure
result: 17 - (2, 'grip0')->(2, 'drawer_bot') = fail
======================= terminated 4: Stop called from other agent  (1.8/30.0) ===============================
[ERROR] Non-joined subprocesses: [13]
========================== FINISHED (2.0 / 30.0 s) ==============================]


In [12]:
snode_schedule = tplan.get_best_schedule()
ppline.play_schedule(snode_schedule)

(2, 'drawer_bot')->(2, 'grip0')
(2, 'grip0')->(1, 'grip0')
(1, 'grip0')->(1, 'drawer_bot')
(1, 'drawer_bot')->(1, 'drawer_bot')
